##  Creating a new directory in the dbfs 

In [0]:
dbutils.fs.mkdirs("dbfs:/external_data/")

**Check that the directory now exists**

In [0]:
dbutils.fs.ls("dbfs:/")

## Run another notebook

In [0]:
%run ../Databricks-Certified-Data-Engineer-Associate/Includes/Copy-Datasets

## Explore catalogs and schema information

**Know the schema and catalog in which you are standing**

In [0]:
%sql
select current_schema()

In [0]:
%sql
select current_catalog();

**Know the managed location of a catalog**

In [0]:
%sql
describe catalog extended workspace;

**Describe a schema**

In [0]:
%sql
describe schema extended workspace.default;

**Show all catalogs**

In [0]:
%sql
show catalogs;

**Show schemas in a catalog**

In [0]:
%sql
show schemas in workspace

## Create table using external data sources

In some cases, a external table can be created to use it as a reference in the project but at the same time, not creating a delta table at all.

In [0]:
%sql
create table hive_metastore.external_data.json_orders
using json
location 'dbfs:/external_data/bookstore/orders-json-raw/*.json';

For big tables, a temporary view is recomended and then load the data into a delta table

In [0]:
%sql
Create temp view json_orders_view 
using json 
options (path 'dbfs:/external_data/bookstore/orders-json-raw/*.json');


In [0]:
%sql
Create table hive_metastore.external_data.json_orders_big as
select *
from json_orders_view

## Querying files directly depending of its location

**JSON files**

In [0]:
%sql
select * from json.`dbfs:/external_data/bookstore/orders-json-raw/*.json`

**CSV files**

In [0]:
csv_file_df = spark.read.option("header", "true").option("delimiter", ";")\
    .csv("dbfs:/external_data/bookstore/books-csv/*.csv")
csv_file_df.show()

**Parquet files**

In [0]:
%sql
select * from parquet.`dbfs:/external_data/bookstore/orders/`

## Incremental inserts

API Call just to generate a sample json data

In [0]:
import requests
import json

response = requests.get( 'https://jsonplaceholder.typicode.com/posts/' )
json_data = response.json()
print(json.dumps(json_data, indent=4))

Writing the json into a delta table

In [0]:
json_df = spark.read.json(spark.sparkContext.parallelize(json_data))
json_df[(json_df.id <= 50)].write.format("delta") \
    .mode("overwrite") \
    .saveAsTable(f"workspace.external_data.json_sample_incremental")

Using the original df to do the incremental loading to the delta table

In [0]:
from delta.tables import DeltaTable

json_deltaTable = DeltaTable.forName(spark,"workspace.external_data.json_sample_incremental")

json_deltaTable.alias("oldData").merge(
    json_df.alias("newData"),
    "oldData.id = newData.id"
).whenNotMatchedInsert( values = 
                       { "body" : "newData.body", "id" : "newData.id", "title" : "newData.title", "userId" : "newData.userId" }
).execute()

In [0]:
%sql
describe history workspace.external_data.json_sample_incremental

Check the versions to see the incremental loading

In [0]:
%sql
select * 
from workspace.external_data.json_sample_incremental
version as of 0;

In [0]:
%sql
select * 
from workspace.external_data.json_sample_incremental
version as of 1;

## Handling string json data

Creating the table from a JSON statement

In [0]:
%sql

create or replace table workspace.external_data.customers as
SELECT *
FROM json.`dbfs:/external_data/bookstore/customers-json/`

With :, you can handle the different levels of a json string row

In [0]:
%sql
select customer_id, profile:first_name, profile:address:country
from workspace.external_data.customers


**Changing a json string column to a struct column**

In [0]:
%sql

create temp view view_json_to_struct as
select customer_id, from_json( profile , schema_of_json('{"first_name":"Dniren","last_name":"Abby","gender":"Female","address":{"street":"768 Mesta Terrace","city":"Annecy","country":"France"}}')) as profile_struct
from workspace.external_data.customers

Querying specific parts of the struct

In [0]:
%sql

select customer_id, profile_struct.first_name, profile_struct.address.country
from view_json_to_struct

Changing the struct to column

In [0]:
%sql
select customer_id, profile_struct.*
from view_json_to_struct

## Handling array columns

In [0]:
%sql
create or replace table workspace.external_data.orders as
SELECT *
FROM parquet.`dbfs:/external_data/bookstore/orders/`

**Splitting an array column into separate rows**

In [0]:
%sql
Select order_id, customer_id, explode( books ) as book
from workspace.external_data.orders

**Collect an array of unique values from other fields**

In [0]:
%sql

select customer_id,
collect_set( order_id ) as orders_set,
collect_set( books.book_id ) as book_set
from workspace.external_data.orders
group by customer_id

**Flatten an array ( collecting unique elements )
**

In [0]:
%sql

select customer_id,
collect_set( order_id ) as orders_set,
array_distinct( flatten( collect_set( books.book_id ) ) ) as book_set
from workspace.external_data.orders
group by customer_id

**Filter arrays within fields**

In [0]:
%sql

select order_id,
books,
filter ( books, i -> i.quantity >= 2 ) as multiple_copies
from workspace.external_data.orders

**Apply transformations to array fields**

In [0]:
%sql

select order_id,
books,
transform ( books, i -> case when i.quantity >= 2 then i.subtotal * 0.9 else i.subtotal end ) as subtotal_after_discount
from workspace.external_data.orders

## Creating customized sql functions for sparksql

SQL Based functions

In [0]:
%sql
create or replace function workspace.external_data.get_url( email string ) 
returns string

return concat( "https://www.", split(email , "@")[1] )

Python based functions

In [0]:
def poisson_sample(lambda_value):
  import numpy 
  return numpy.random.poisson(lambda_value)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

poisson_udf = udf(poisson_sample, IntegerType())
